## Dataloading: datasets and dataloaders

Training = Dataset and Dataloader + Model + Loss and Optimizer

## Dataset class: defining your own dataset.

`torch.utils.data.Dataset` is a python module of a class representing a dataset. There are some predefined datasets (MNIST, CIFAR,...).
A custom dataset should inherit `Dataset` and override at least the two following methods:

  - `__len__` so that `len(dataset)` returns the size of the dataset.
  - `__getitem__` to support the indexing such that `dataset[i]` can be used to get the *i*th sample of the dataset.

## Dataloader class: iterating through the dataset

By using a simple `for` loop to iterate over the data, we are missing out on:

  -  Batching the data
  -  Shuffling the data
  -  Load the data in parallel using multiprocessing workers.

`torch.utils.data.DataLoader` is an iterator which provides all these features.

## Example (1)

Remember the cats and dogs example where we create two datasets, one for the training and one for the validation: (using standard scalar to make your own)
```
from torchvision import transforms,datasets

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

imagenet_format = transforms.Compose([transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      normalize])

dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              imagenet_format) for x in ['train', 'valid']}
```

Hence `len(dsets['train'])` returns `23000`, i.e. the number of images in the training set, and more precisely, the number of files located at `data_dir/train/`.

Recall that `data_dir/train/` (and similarly `data_dir/valid/`) is split in two folders: `cats/` and `dogs/`, you can check that each of these folders have `11500` images with: `ls | wc -l`

You can recover the classes with `dsets['train'].classes` which returns `['cats', 'dogs']`
These are features of the [`torchvision.datasets.ImageFolder`](https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html) Module.




* **Q**: what returns `dsets['train'][0]`?
* **A**: a tuple containing a tensor and a label.

To obtain a dataloader for the training set:
```
train_loader = torch.utils.data.DataLoader(dsets['train'],
                                           batch_size=64,
                                           shuffle=True,
                                           num_workers=6)
```

Then, you can use it as follows:
```
for input, label in train_loader:
    output = model(input)
    loss = loss_fn(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    ....
```

## Example (2)

Assume now you have a large 1B row CSV file, you can create your custom dataset to iterate every batch of 10,000 rows.
```
import torch
from torch.utils.data import Dataset, DataLoader

import pandas as pd

class MyDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file, header=None).values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx, :], dtype=torch.float)

batch_size = 10000
dataset = MyDataset("path/to/my/file.csv")
dataloader = DataLoader(dataset, 
                        batch_size=batch_size,
                        shuffle=True, 
                        num_workers=4)
```

## To know more about dataloading


* Take a look at the [PyTorch tutorial](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) and this [Paperspace blog](https://blog.paperspace.com/dataloaders-abstractions-pytorch/).
* A [more detailed example](https://machinelearningmastery.com/training-a-pytorch-model-with-dataloader-and-dataset/)